In [35]:
# For this query, I used all of the data I collected to gather a deeper insight. 

# Business Question:
# Which NBA teams are overperforming or underperforming relative to their online popularity?
# This supports decisions on where marketing and engagement budgets should be focused.

# Query Summary:
# - Ranked teams by total wins since 2000 (`performance_rank`)
# - Ranked teams by total social media followers (`social_rank`)
# - Calculated `rank_gap` to identify over/underexposure
    # - Positive gap: High performance, low popularity
    # - Negative gap: Popular brand, lower win history

import pandas as pd
from sqlalchemy import create_engine

USER = "tracy_mcgrady"
PASSWORD = "shaq_attack123"
HOST = "nba-project.c7g8wiquaqg7.us-east-1.rds.amazonaws.com"
PORT = "5432"
DATABASE = "tmac"
SCHEMA = "sql_project"

engine = create_engine(
    f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}?sslmode=require"
)

print("Connected to database!")

# SQL query
query = f'''
WITH long_term_performance AS (
    SELECT 
        team_name,
        SUM(wins) AS total_wins
    FROM sql_project.nba_standings_data
    WHERE season >= '2000-2001'
    GROUP BY team_name
),
performance_rank AS (
    SELECT 
        team_name,
        total_wins,
        RANK() OVER (ORDER BY total_wins DESC) AS performance_rank
    FROM long_term_performance
),
social_media AS (
    SELECT 
        t.team_full_name AS team_name,
        COALESCE(r.reddit_subscribers, 0) +
        COALESCE(CAST(REPLACE(x.x_followers_millions::TEXT, ',', '')::FLOAT * 1e6 AS BIGINT), 0) +
        COALESCE(CAST(REPLACE(y.subscribers_thousands::TEXT, ',', '')::FLOAT * 1e3 AS BIGINT), 0) AS total_social_followers
    FROM sql_project.nba_teams t
    LEFT JOIN sql_project.reddit_fan_metrics r ON t.team_full_name = r.team_name
    LEFT JOIN sql_project.x_statistics x ON t.team_full_name = x.team_name
    LEFT JOIN sql_project.nba_youtube_statistics y ON t.team_full_name = y.team_name
),
social_ranked AS (
    SELECT 
        team_name,
        total_social_followers,
        RANK() OVER (ORDER BY total_social_followers DESC) AS social_rank
    FROM social_media
),
joined AS (
    SELECT 
        p.team_name,
        p.performance_rank,
        p.total_wins,
        s.social_rank,
        s.total_social_followers,
        (p.performance_rank - s.social_rank) AS rank_gap
    FROM performance_rank p
    JOIN social_ranked s ON p.team_name = s.team_name
)

SELECT * FROM joined
ORDER BY performance_rank ASC;
'''

# Run query
try:
    df = pd.read_sql(query, engine)
    print(f"Query successful — {len(df)} rows returned.")
    display(df if not df.empty else "No results. Check data consistency.")
except Exception as e:
    print("Query failed:", e)

# Insights:
# - Indiana Pacers, Denver Nuggets, and Utah Jazz are winning consistently but lack online visibility — high potential for marketing focus.
# - Thunder, Bulls, and Nets have strong digital engagement despite weaker win records — suggesting brand power or star appeal drives their popularity.

# Recommendation:
# - Increase social campaigns or influencer outreach for underexposed high-performing teams.
# - Leverage strong digital brands (e.g. Bulls, Warriors) for merch and fan growth despite inconsistent wins.

# Prediction:
# - Rebalancing attention toward overlooked high-performing teams may boost jersey sales, attendance, and engagement.
# - Sustained investment in popular underperformers can still pay off if driven by players, legacy, or market size.


Connected to database!
Query successful — 29 rows returned.


,team_name,performance_rank,total_wins,social_rank,total_social_followers,rank_gap
0,San Antonio Spurs,1,1209.0,11,3752717,-10
1,Boston Celtics,2,1115.0,5,5933844,-3
2,Dallas Mavericks,3,1092.0,9,4080777,-6
3,Denver Nuggets,4,1067.0,17,2947095,-13
4,Indiana Pacers,5,1057.0,25,2172193,-20
5,Milwaukee Bucks,6,1027.0,18,2860809,-12
6,Los Angeles Lakers,7,1026.0,1,14467188,6
7,Miami Heat,8,1023.0,3,6744911,5
8,Houston Rockets,9,1004.0,6,4493507,3
9,Phoenix Suns,11,986.0,19,2750453,-8
